# Amazon Lab 2
Paine(HE Peiwei, 54471962)

# Research Question: the most controversial digital music product

First, open the raw data of Digital Music on Amazon, and create lists for reviews and products seperately.

In [2]:
import json
import gzip

In [3]:
reviewers,products ={},{}
fn = "Digital_Music_5.json"
reviews = []
for line in open(fn):
    review = eval(line)
    reviews.append(review)

In [4]:
product_ids = set()
for review in reviews:
    product_id = review['asin']
    product_ids.add(product_id)
print(list(product_ids)[:5])

['B0000047RY', 'B0000E1ALR', 'B002N1AE8W', 'B000QVTMUM', 'B000WGWCLM']


In [5]:
products = {}
fn = "meta_Digital_Music.json"
for line in open(fn):
    d = eval(line)
    if d['asin'] in product_ids:
        products[d['asin']] = d
print(len(products))

3568


In [6]:
p_reviews = {}
for review in reviews:
    asin = review['asin']
    if asin not in p_reviews:
        p_reviews[asin] = [review]
    else:
        p_reviews[asin].append(review)

Second, define a new function called controversiality. Controversiality means to count the number of good (5 stars) and bad (1 or 2 stars) reviews, and take the lowest of good and bad. By sorting the products, I find that the album 50 Cent-Get Rich Or Die Tryin(2003) is the most controversial music product, and I check more detailed features of this product, such as the score distribution, the total reviews number, examples of positive and negative comments.

In [30]:
def controversiality(asin):
    reviews = p_reviews[asin]
    counts = count_scores(reviews)
    negative = counts[1] + counts[2]
    positive = counts[5]
    return min(negative, positive)
print(controversiality(most))

48


In [15]:
ids = sorted(p_reviews, key=controversiality, reverse=True)
controv = ids[0]
print("Most controversial:", controv, "(", products[controv]['title'],"): ", controversiality(most))

Most controversial: B000084T18 ( Get Rich Or Die Tryin ):  48


In [39]:
def n_reviews(asin):
    reviews = p_reviews[asin]
    return len(reviews)
print(n_reviews("B000084T18"))
print(count_scores(p_reviews[controv]))

271
{1: 61, 2: 14, 3: 33, 4: 63, 5: 100}


In [17]:
negatives = [review for review in p_reviews[controv] if review['overall'] == 1.0]
positives = [review for review in p_reviews[controv] if review['overall'] == 5.0]
print("Negative example:", negatives[0]['reviewText'])
print("\nPositive example:", positives[0]['reviewText'])

Negative example: If I was you Id click on Dr Dre and get both the Chronic albums instead.other albums that are betterMakaveli-the Don KiluminatiSnoop Doggy Dogg-DoggystyleNWA-Straight outta ComptonTha Dogg Poung-Dogg FoodIce Cube- Death CertificateNotorious BIG- Ready to DieListen to those and then compare if fiddy is a lyricists.

Positive example: I brought this album when it first came out in 03 and it still sounds like fire today. It's a pretty long album and the subjects are limited (guns, drugs, sex, killing) but the beats are hot and there are no bad songs.My Favorites1. Patiently Waiting w/Eminem2. Many Men3. In Da Club4. Heat5. If I Can't6. Blood Hound w/ Young Buck7. Like My Style w/ Tony Tayo8. 21 QuestionsI think this is one of the best rap albums of the last 10 years and will pass it on to a friend


Third, I define a function of getting the most frequent words in positive and negative reviews, I test this function with keyword "cent", which represents the rapper "50 Cent", and the result shows 50 Cent is quite controversial. After that, I get the most frequent words in positive and negative comments, and stop some meanningless words mannually. The result is quite interesting that "50 Cent" is mentioned by consumers both in postive and negative reviews, and some other words like "Eminem", "Dr.Dre", "Rap" also show a popular musical flavour in Rap music.

In [44]:
import re
def count_words(texts):
    counts = {}
    for text in texts:
        text = text.lower()
        words = re.split("\W+", text)
        for word in words:
            counts[word] = counts.get(word, 0) + 1
    return counts

In [45]:
positive_words = count_words([review['reviewText'] for review in positives])
negative_words = count_words([review['reviewText'] for review in negatives])
positive_words.get('cent'), negative_words.get('cent')

(102, 62)

In [27]:
def get_value(item):
    return item[1]
positive_words_list = sorted(positive_words.items(), key=get_value, reverse=True)

In [28]:
stopwords = {"the", "a", "i", "it", "is", "of", "to", "this", "and", "in", "","don","has","p","have","be","one","for", "t","s","he","on","that","you","are","was","but","with","all","not","if","my","me","as","up","or","can","what","da"}
positive_words_list = [(w, n) for (w, n) in positive_words_list if w not in stopwords]
for word, n in positive_words_list[:20]:
             print(repr(word), n)

'50' 292
'album' 245
'like' 170
'5' 168
'his' 152
'song' 148
'cent' 102
'10' 99
'great' 95
'cd' 93
'get' 87
'good' 79
'eminem' 74
'just' 66
'club' 65
'best' 65
'dre' 61
'many' 60
'rich' 60
'rap' 59


In [29]:
negative_words_list = sorted(negative_words.items(), key=get_value, reverse=True)
negative_words_list = [(w, n) for (w, n) in negative_words_list if w not in stopwords]
for word, n in negative_words_list[:20]:
    print(repr(word), n)

'50' 113
'album' 69
'rap' 62
'cent' 62
'his' 62
'like' 59
'about' 55
'they' 48
'music' 47
'because' 36
'just' 35
'only' 34
'why' 33
'no' 33
'when' 32
'do' 31
'out' 31
'cd' 31
'even' 31
'your' 30
